In [113]:
import glob
import pandas as pd

In [114]:
basedir = '/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output'
pat = 'retina_*_cells_all_*_batch_donor_dataset_cell.type_2022/metrics/unscaled/HVG.*K/*.csv'

In [115]:
metrics_paths = glob.glob(basedir + "/" + pat)

In [116]:
print('here...')

here...


In [117]:
import os
def is_non_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

df = []
for i, p in enumerate(metrics_paths):
    f = p.split('/')[-1]
    if i % 50 == 0:
        print(i, len(metrics_paths))
    if 'backup' in p and not 'backup_subset' in f: # ignore backup files
        continue
    if not is_non_zero_file(p):
        continue
    # print(f, p)
    # print(p)
    df2 = pd.read_csv(p, index_col=0)
    assert df2.shape[1] == 1
    
    
    
    method = df2.columns[0]
    df2['method'] = method
    df2.index.name = 'metric'
    df2 = df2[~pd.isnull(df2[method])].reset_index().rename(columns={method: 'value'})
    
    if 'isolatedlabelssilhouette' in f:
        df2 = df2[df2['metric'] == 'isolated_label_silhouette']
        assert df2.shape[0] == 1
    
    # print(hvg)
    df2['hvg'] = p.split('/')[-2]
    df2['rep'] = p.split('_')[-2]
    df2['sample'] = p.split('/')[-5].split('_')[1]
    df2['tech_group'] = p.split('/')[-5].split('_')[4]
    df2['metrics_subset'] = 'both' if not 'subset_sc' in f and not 'subset_sn' in f else 'sn' if 'subset_sn' in f else 'sc' if 'subset_sc' in f else None
    df2['filename'] = f
    df2['path'] = p
    # print(df)
    df.append(df2)
    
df = pd.concat(df).reset_index(drop=True)


0 746
50 746
100 746
150 746
200 746
250 746
300 746
350 746
400 746
450 746
500 746
550 746
600 746
650 746
700 746


### Prepare all features input


In [124]:
for v, df_metrics in df.groupby(['sample', 'tech_group', 'hvg', 'metrics_subset']):
    sample, tech, hvg, metrics = v
    print(sample, tech, metrics, df_metrics.shape)
    
    a = df_metrics.path.str.replace('/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output', '')
    new_index = []
    for ri, r in df_metrics.iterrows():
        # print(r['filename'])
        new_index.append(a[ri].replace('/' + r['filename'], '') + '/' + r['method'])
        # print(new_index[-1])
        # assert False
    # assert False
    df_metrics['method_table'] = new_index
    hm = df_metrics.pivot('method_table', 'metric', 'value')
    output_filename = '/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/%s_%s_%s_subset_%s.csv' % (sample, tech, hvg, metrics) 
    # print(hm.shape)
    # print(output_filename)    
    hm.to_csv(output_filename)
    # break


500 sn both (56, 10)


/tmp/ipykernel_295131/2731316255.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  a = df_metrics.path.str.replace('/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output', '')


500 sn both (44, 10)
500 sn+sc both (118, 10)
500 sn+sc sc (54, 10)
500 sn+sc sn (54, 10)
500 sn+sc both (44, 10)
500 sn+sc sc (44, 10)
500 sn+sc sn (44, 10)
all sc both (5, 10)
all sc both (13, 10)
all sc sc (20, 10)
all sc sn (44, 10)
all sn both (1, 10)
all sn both (2, 10)


### Prepare table with sn subsets

In [125]:
for sample, df_sample in df.groupby(['sample']):
    # if sample != '500':
    #     continue
    for hvg, df_hvg in df_sample.groupby(['hvg']):      
        df1 = df_hvg[(df_hvg['tech_group'] == 'sn')]
        df2 = df_hvg[(df_hvg['tech_group'] == 'sn+sc') & (df_hvg['metrics_subset'] == 'sn')]
        
        df_metrics = pd.concat([df1, df2])        
        print(sample, hvg)
        print(df_metrics['tech_group'].value_counts())
        print('')
        
        # assert False

        # print(sample, tech, hvg, metrics, df_metrics.shape)

        # continue

        a = df_metrics.path.str.replace('/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output', '')
        new_index = []
        for ri, r in df_metrics.iterrows():
            # print(r['filename'])
            k = a[ri].replace('/' + r['filename'], '') + '/' + r['method']
            k = k.replace('HVG.1K', 'HVG.1K.' + r['tech_group']).replace("+", "")
            k = k.replace('HVG.3K', 'HVG.3K.' + r['tech_group']).replace("+", "")
            new_index.append(k)
            # print(new_index[-1])
            # assert False
        # assert False
        df_metrics['method_table'] = new_index
        df_metrics['method_table'] = df_metrics['method_table'] # + '_' + df_metrics['tech_group'] + '_' + df_metrics['metrics_subset']
        hm = df_metrics.pivot('method_table', 'metric', 'value')
        output_filename = '/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/%s_%s_sncombined.csv' % (sample, hvg) 
        print(output_filename)
        # print(hm.shape)
        # print(output_filename)    
        hm.to_csv(output_filename)
        # break


500 HVG.1K
sn       56
sn+sc    54
Name: tech_group, dtype: int64

/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/500_HVG.1K_sncombined.csv
500 HVG.3K
sn       44
sn+sc    44
Name: tech_group, dtype: int64

/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/500_HVG.3K_sncombined.csv


/tmp/ipykernel_295131/1626089973.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  a = df_metrics.path.str.replace('/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output', '')


all HVG.1K
sn    1
Name: tech_group, dtype: int64

/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/all_HVG.1K_sncombined.csv
all HVG.3K
sn    2
Name: tech_group, dtype: int64

/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/all_HVG.3K_sncombined.csv


### Prepare table with sc subsets

In [110]:
for sample, df_sample in df.groupby(['sample']):
    if sample != '500':
        continue
    for hvg, df_hvg in df_sample.groupby(['hvg']):      
        df1 = df_hvg[(df_hvg['tech_group'] == 'sc')]
        df2 = df_hvg[(df_hvg['tech_group'] == 'sn+sc') & (df_hvg['metrics_subset'] == 'sc')]
        
        df_metrics = pd.concat([df1, df2])        
        print(sample, hvg)
        print(df_metrics['tech_group'].value_counts())
        print('')
        
        # assert False

        # print(sample, tech, hvg, metrics, df_metrics.shape)

        # continue

        a = df_metrics.path.str.replace('/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output', '')
        new_index = []
        for ri, r in df_metrics.iterrows():
            # print(r['filename'])
            k = a[ri].replace('/' + r['filename'], '') + '/' + r['method']
            k = k.replace('HVG.1K', 'HVG.1K.' + r['tech_group']).replace("+", "")
            k = k.replace('HVG.3K', 'HVG.3K.' + r['tech_group']).replace("+", "")
            new_index.append(k)
            # print(new_index[-1])
            # assert False
        # assert False
        df_metrics['method_table'] = new_index
        df_metrics['method_table'] = df_metrics['method_table'] # + '_' + df_metrics['tech_group'] + '_' + df_metrics['metrics_subset']
        hm = df_metrics.pivot('method_table', 'metric', 'value')
        output_filename = '/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/%s_%s_sccombined.csv' % (sample, hvg) 
        print(output_filename)
        # print(hm.shape)
        # print(output_filename)    
        hm.to_csv(output_filename)
        # break


500 HVG.1K
sn+sc    54
Name: tech_group, dtype: int64

/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/500_HVG.1K_sccombined.csv
500 HVG.3K
sn+sc    44
Name: tech_group, dtype: int64

/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output/500_HVG.3K_sccombined.csv


/tmp/ipykernel_295131/123308798.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  a = df_metrics.path.str.replace('/lustre/groups/ml01/workspace/ignacio.ibarra/theislab/retina/data/integration_oct_2022/output', '')
